In [466]:
import pandas as pd
import numpy as np
from datetime import datetime
import math
from pytz import timezone

In [467]:
trade_debitors_2018 = pd.read_csv("/Users/sumit.khaitan/Desktop/trade_debitor_2018.csv")
trade_debitors_2019 = pd.read_csv("/Users/sumit.khaitan/Desktop/trade_debitor_2019.csv")
trade_debitors_2018.head()

,customer,customer_group,01/18(total),01/18(not-overdue),01/18(overdue),01/18(0-30),01/18(30-60),01/18(60-90),01/18(90-120),01/18(120-180),...,12/18(total),12/18(not-overdue),12/18(overdue),12/18(0-30),12/18(30-60),12/18(60-90),12/18(90-120),12/18(120-180),12/18(180-365),12/18(>365)
0,99818804,DIRECT SALES,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0
1,94733770,SUNDRY SALES OTHER,45474.54,18319.96,27154.58,0.00,0.0,0.00,0.0,0.00,...,-3587.16,0.0,-3587.16,0.0,0.00,0.0,0.00,14732.8,-18319.96,0.0
2,91608738,SUNDRY SALES OTHER,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0
3,90103311,SUNDRY SALES DAMAGE CLAIMS,40848.82,0.00,40848.82,5800.63,0.0,16514.12,0.0,3844.74,...,4110.84,0.0,4110.84,0.0,3054.52,0.0,1312.34,0.0,-256.02,0.0
4,90009763,SUNDRY SALES DAMAGE CLAIMS,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0


In [481]:
def get_debitors(trade_debitors, output_file) :
    f = open(output_file, "w")
    
    for i, j in trade_debitors.iterrows():
        customer = j['customer']
        customer_group = j['customer_group']
        
        if(customer_group == 0):
            customer_group = "unknown"
            
        customer_group = customer_group.replace(" ","_")
        
        for h in trade_debitors.columns.tolist():
            if(h == "customer" or h == "customer_group"):
                continue
            date = h.split("(")[0].replace("/18","/2018").replace("/19","/2019")
            epoc_date = math.trunc(datetime.strptime(date, "%m/%Y").replace(tzinfo=timezone('UTC')).timestamp())
            tag = h.split("(")[1].replace(")","")
            value = j[h]
            
            if(value != 0):
                #print(date, epoc_date, tag, customer, customer_group)
                path = "cash.debtors.trade_debtors_customer." + customer_group + "." + str(customer)
                if(tag == 'total' or tag == 'not-overdue' or tag == 'overdue'):
                    path = path + "." + tag
                else:
                    path = path + ".overdue_bucket." + tag
                    
                #print(path.lower())
                graphite_push = 'echo "' + path.lower() + " " + str(value) + " " + str(epoc_date) + '" | nc localhost 8003'
                #print(graphite_push)
                f.write(graphite_push+"\n")
                
    f.close()
    print("done")
        
get_debitors(trade_debitors_2018,'/Users/sumit.khaitan/Desktop/2018_debitor.sh')
get_debitors(trade_debitors_2019,'/Users/sumit.khaitan/Desktop/2019_debitor.sh')

done
done


In [469]:
gl_data = pd.read_csv("GL mapping.csv").rename(columns={"GL account code": "gl_code"})
gl_data.head(10)

,gl_code,GL account description,Level 1,Level 2
0,13301010,Trade Debtors Swiss Based,Debtors,Trade Debtors - Customer
1,13301100,Trade Debtors Currency Revaluation,Debtors,Trade Debtors
2,13301110,Trade DebtorsNon- Swiss Based,Debtors,Trade Debtors - Customer
3,13308010,Trade Debtors Reclassification Swiss Based,Debtors,Trade Debtors
4,13308110,Trade Debtors Reclassification non-Swiss Based,Debtors,Trade Debtors
5,13384001,Trade Debtors - debit balances,Debtors,Trade Debtors
6,13301204,AR Trad - Debtors Upfield,Debtors,Trade Debtors - Customer
7,13385010,AR Trade - TSA Trade Receivables Reclas,Debtors,Trade Debtors
8,13385001,Current Trade Receivables - TSA Trade R,Debtors,Trade Debtors - Customer
9,13308888,P13 Curr Trade Receivables - Reclassifi,Debtors,Trade Debtors


In [470]:
print(gl_data['Level 1'].unique())
print(gl_data['Level 2'].unique())
print(gl_data['gl_code'].unique())

['Debtors' 'Creditors']
['Trade Debtors - Customer' 'Trade Debtors' 'Other Prepayments'
 'Hedging Debtors' 'VAT Debtors' 'Payroll Debtors'
 'Trade Creditors - Supplier' 'Trade Creditors' 'GRNI' 'Consignment'
 'V105 & Bankclearing' 'Deliver' 'VAT Creditors' 'Indirect Tax'
 'Payroll Creditors' 'Hedging Creditors' 'Other Creditors']
[13301010 13301100 13301110 13308010 13308110 13384001 13301204 13385010
 13385001 13308888 13399010 13501001 13501010 13501020 13501030 13501040
 13501090 13501100 13501750 13512001 13512010 13501110 13512057 13405001
 13308001 13501444 13512052 13512977 13512054 13408040 13460001 13466001
 13461001 13462001 13407002 13407001 13407003 13407252 13407253 13407255
 13407258 13407259 13407274 13407282 13407023 13407099 13407152 13407153
 13407154 13407155 13407156 13407157 13407158 13407159 13407160 13407161
 13407166 13407167 13407169 13407173 13407174 13407182 13407186 13407860
 13407187 13407706 13407725 13407758 13407761 13407823 13407857 13407286
 13407287 1

In [471]:
gl_modified_dataframe = [[]]
i=0
for i, j in gl_data.iterrows():
    level_1 = j['Level 1']
    level_2 = j['Level 2']
    gl_code = j['gl_code']
    
    if(level_1 == 'Debtors'):
        if(level_2 != 'Trade Debtors - Customer'):
            level_2 = 'other_Debtors'
        else:
            level_2 = 'trade_debtors_customer'
    else:
        if(level_2 != 'Trade Creditors - Supplier'):
            level_2 = 'other_creditors'
        else:
            level_2 = 'trade_creditors_supplier'
            
    #print(gl_code, level_1, level_2)
    li = [gl_code, level_1, level_2]
    #print(li)
    gl_modified_dataframe.insert(i,li)
    i = i+1

#print(gl_modified_dataframe)
df = pd.DataFrame(gl_modified_dataframe, columns = ['gl_code', 'level_1','level_2']).fillna(0)
df.head(10)

,gl_code,level_1,level_2
0,13301010.0,Debtors,trade_debtors_customer
1,13301100.0,Debtors,other_Debtors
2,13301110.0,Debtors,trade_debtors_customer
3,13308010.0,Debtors,other_Debtors
4,13308110.0,Debtors,other_Debtors
5,13384001.0,Debtors,other_Debtors
6,13301204.0,Debtors,trade_debtors_customer
7,13385010.0,Debtors,other_Debtors
8,13385001.0,Debtors,trade_debtors_customer
9,13308888.0,Debtors,other_Debtors


In [472]:
raw_data = pd.read_csv("/Users/sumit.khaitan/Desktop/Raw data Working Capital.csv").fillna(0)
raw_data = raw_data.drop(['01/2017','02/2017','03/2017','04/2017','05/2017','06/2017','07/2017','08/2017','09/2017','10/2017','11/2017','12/2017'],axis=1)
raw_data.loc[:, raw_data.columns != 'gl_code'] = raw_data.loc[:, raw_data.columns != 'gl_code'].apply(lambda x: x*1000000)
raw_data.head(10)

,gl_code,01/2018,02/2018,03/2018,04/2018,05/2018,06/2018,07/2018,08/2018,09/2018,...,03/2019,04/2019,05/2019,06/2019,07/2019,08/2019,09/2019,10/2019,11/2019,12/2019
0,13301010.0,7510000.0,7840000.0,15690000.0,15140000.0,8420000.0,8330000.0,9220000.0,13190000.0,18000000.0,...,10920000.0,10740000.0,12240000.0,14020000.0,14220000.0,12450000.0,12790000.0,13020000.0,13070000.0,12620000.0
1,13301100.0,-130000.0,30000.0,10000.0,-30000.0,10000.0,-30000.0,0.0,-10000.0,0.0,...,100000.0,50000.0,30000.0,0.0,-30000.0,-10000.0,20000.0,50000.0,50000.0,50000.0
2,13301110.0,45600000.0,42280000.0,41750000.0,37980000.0,38160000.0,40980000.0,41820000.0,38110000.0,39670000.0,...,37950000.0,41640000.0,46620000.0,37650000.0,36350000.0,33890000.0,36020000.0,36300000.0,32390000.0,39590000.0
3,13308010.0,890000.0,660000.0,530000.0,420000.0,8280000.0,3030000.0,3570000.0,2630000.0,440000.0,...,150000.0,160000.0,160000.0,20000.0,20000.0,20000.0,20000.0,-50000.0,-20000.0,-40000.0
4,13308110.0,-680000.0,-480000.0,220000.0,600000.0,180000.0,310000.0,110000.0,140000.0,40000.0,...,220000.0,540000.0,490000.0,560000.0,380000.0,140000.0,120000.0,190000.0,190000.0,140000.0
5,13384001.0,10090000.0,8500000.0,13010000.0,7070000.0,9740000.0,10750000.0,14330000.0,12990000.0,14440000.0,...,8320000.0,8120000.0,6810000.0,4970000.0,4610000.0,4770000.0,4810000.0,4160000.0,4720000.0,3420000.0
6,13301204.0,0.0,0.0,0.0,0.0,0.0,0.0,620000.0,740000.0,0.0,...,260000.0,420000.0,150000.0,170000.0,210000.0,140000.0,150000.0,190000.0,250000.0,190000.0
7,13385010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1850000.0,1850000.0,1860000.0,50000.0,60000.0,50000.0,50000.0,50000.0,50000.0,1120000.0
8,13385001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100000.0,2730000.0,2690000.0,30000.0,650000.0,5730000.0,4880000.0,4800000.0,1790000.0,2020000.0
9,13308888.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-330000.0,-350000.0,250000.0,570000.0,200000.0,180000.0,150000.0,130000.0,130000.0,20000.0


In [473]:
gl_join = raw_data.merge(df, on='gl_code',how='outer')
gl_join.head()

,gl_code,01/2018,02/2018,03/2018,04/2018,05/2018,06/2018,07/2018,08/2018,09/2018,...,05/2019,06/2019,07/2019,08/2019,09/2019,10/2019,11/2019,12/2019,level_1,level_2
0,13301010.0,7510000.0,7840000.0,15690000.0,15140000.0,8420000.0,8330000.0,9220000.0,13190000.0,18000000.0,...,12240000.0,14020000.0,14220000.0,12450000.0,12790000.0,13020000.0,13070000.0,12620000.0,Debtors,trade_debtors_customer
1,13301100.0,-130000.0,30000.0,10000.0,-30000.0,10000.0,-30000.0,0.0,-10000.0,0.0,...,30000.0,0.0,-30000.0,-10000.0,20000.0,50000.0,50000.0,50000.0,Debtors,other_Debtors
2,13301110.0,45600000.0,42280000.0,41750000.0,37980000.0,38160000.0,40980000.0,41820000.0,38110000.0,39670000.0,...,46620000.0,37650000.0,36350000.0,33890000.0,36020000.0,36300000.0,32390000.0,39590000.0,Debtors,trade_debtors_customer
3,13308010.0,890000.0,660000.0,530000.0,420000.0,8280000.0,3030000.0,3570000.0,2630000.0,440000.0,...,160000.0,20000.0,20000.0,20000.0,20000.0,-50000.0,-20000.0,-40000.0,Debtors,other_Debtors
4,13308110.0,-680000.0,-480000.0,220000.0,600000.0,180000.0,310000.0,110000.0,140000.0,40000.0,...,490000.0,560000.0,380000.0,140000.0,120000.0,190000.0,190000.0,140000.0,Debtors,other_Debtors


In [474]:
f = open("/Users/sumit.khaitan/Desktop/gl_data.sh", "w")

for i, j in gl_join.iterrows():
    gl_code = math.trunc(j['gl_code'])
    if(gl_code == 0):
        continue
    level_1 = j['level_1']
    level_2 = j['level_2']
    for k in gl_join.columns.tolist():
        if(k == 'gl_code' or k == 'level_1' or k == 'level_2'):
            continue
        epoc_date = math.trunc(datetime.strptime(k, "%m/%Y").replace(tzinfo=timezone('UTC')).timestamp())
        value = math.trunc(j[k])
        if(value != 0):
            path = "cash." + level_1 + "." + level_2 + "." + str(gl_code)
            #print(k, epoc_date, path.lower(), value)
            to_push = 'echo "' + path.lower() + " " + str(value) + " " + str(epoc_date) + '"' + ' | nc localhost 8003'
            f.write(to_push+"\n")
            
f.close
print("done")
        

done


In [475]:
trade_creditors = pd.read_csv("/Users/sumit.khaitan/Desktop/trade_creditors.csv").fillna(0)
print(trade_creditors.shape)
trade_creditors.head(10)
print(trade_creditors['Vendor'].unique())

(3458, 23)
[59000739.   380419. 50282920. ... 50432010. 50485599.        0.]


In [480]:
f = open("/Users/sumit.khaitan/Desktop/creditors_supplier.sh", "w")

for i, j in trade_creditors.iterrows():
    vendor = j['Vendor']
    group = j['Group']
    
    if(vendor == 0):
        continue
    
    for h in trade_creditors.columns.tolist():
        if(h == "Vendor" or h == "Group"):
            continue
        epoc_date = math.trunc(datetime.strptime(h, "%m/%Y").replace(tzinfo=timezone('UTC')).timestamp())
        value = j[h]
        
        if(value != 0):
            #print(vendor, group, h, epoc_date,value)
            path = "cash.creditors.trade_creditors_supplier." + str(group) + "." + str(math.trunc(vendor))
            #print(path)

            graphite_push = 'echo "' + path.lower() + " " + str(math.trunc(value)) + " " + str(epoc_date) + '" | nc localhost 8003'
            f.write(graphite_push + "\n")
            
f.close()
print("done")

done
